In [1]:

import sys
import numpy as np
import ase.io
import ase.units as units
from ase.units import GPa, J, m

from matscipy.fracture_mechanics.clusters import diamond, set_groups
from ase.calculators.lammpslib import LAMMPSlib

import sys

# Fundamental material properties
el  = 'Si'

from ase.build import bulk
from ase.constraints import ExpCellFilter
from ase.optimize import LBFGSLineSearch
from ase.optimize.precon import PreconLBFGS
from matscipy.elasticity import fit_elastic_constants
from ase.optimize import *

In [ ]:
import pyjulip
# calc_asp = pyjulip.ACE1(f"Si_pot_tsvdASP_2500.json")
calc_asp_5000 = pyjulip.ACE1(f"asp1000.json")
# calc_ace = pyjulip.ACE1(f"Si_4_20.json")

/Users/tinatorabi/gits/Results/asp_paper/Si/Si_pots/gp_iter6_sparse9k.xml

In [ ]:
# calc_omp_300 = pyjulip.ACE1(f"Si_pot_ASP_2500rn.json")
calc_omp_300 = pyjulip.ACE1(f"asp1000.json")



In [39]:
def find_surface_energy(symbol, calc, a0, surface, size=(8,1,1), vacuum=10, fmax=0.0001, unit='0.1J/m^2'):

    # Import required lattice builder
    if surface.startswith('bcc'):
        from ase.lattice.cubic import BodyCenteredCubic as lattice_builder
    elif surface.startswith('fcc'):
        from ase.lattice.cubic import FaceCenteredCubic as lattice_builder #untested
    elif surface.startswith('diamond'):
        from ase.lattice.cubic import Diamond as lattice_builder #untested
    ## Append other lattice builders here
    else:
        print('Error: Unsupported lattice ordering.')

    # Set orthogonal directions for cell axes
    import numpy as np
    if surface.endswith('100'):
        directions=[[1,0,0], [0,1,0], [0,0,1]] #tested for bcc
    elif surface.endswith('110'):
        directions=[[1,1,0], [-1,1,0], [0,0,1]] #tested for bcc
    elif surface.endswith('111'):
        directions=[[1,1,1], [-2,1,1],[0,-1,1]] #tested for bcc
    ## Append other cell axis options here
    else:
        print('Error: Unsupported surface orientation.')

    # Make bulk and slab with same number of atoms (size)
    bulk = lattice_builder(directions=directions, size=size, symbol=symbol, latticeconstant=a0, pbc=(1,1,1))
    if surface == 'diamond111':
        sx, sy, sz = bulk.get_cell().diagonal()       
        bulk.translate([sx/(6*size[0]), sy/(4*size[1]),  sz/(12*size[2])])
        bulk.set_scaled_positions(bulk.get_scaled_positions()%1.0)
    cell = bulk.get_cell() ; cell[0,:] *=2 # vacuum along x axis (surface normal)
    slab = bulk.copy() ; slab.set_cell(cell)
    
    # Optimize the geometries
    bulk.calc = calc ; opt_bulk = PreconLBFGS(bulk) ; opt_bulk.run(fmax=fmax)
    slab.calc = calc ; opt_slab = PreconLBFGS(slab, use_armijo=False) ; opt_slab.run(fmax=fmax)
    
    # Find surface energy
    import numpy as np
    Ebulk = bulk.get_potential_energy() ; Eslab = slab.get_potential_energy()
    area = np.linalg.norm(np.cross(slab.get_cell()[1,:],slab.get_cell()[2,:]))
    gamma_ase = (Eslab - Ebulk)/(2*area)

    # Convert to required units
    if unit == 'ASE':
        return [gamma_ase,'ase_units']
    else:
        gamma_SI = (gamma_ase / units.J ) * (units.m)**2
        if unit =='J/m^2':
            return gamma_SI
        elif unit == '0.1J/m^2':
            return 10*gamma_SI # units required for the fracture code
        else:
            print('Error: Unsupported unit of surface energy.')


In [ ]:
from ase import Atoms
from ase.optimize import BFGS
import numpy as np
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.optimize.precon import Exp, PreconLBFGS
from ase.optimize import BFGS

si_bulk = bulk('Si', 'diamond', a=5.44, cubic=True)
mm_pot = calc_asp_5000

from ase.optimize.bfgslinesearch import BFGSLineSearch
dyn = BFGSLineSearch(atoms=si_bulk, trajectory='bfgs_ls.traj', restart='bfgs_ls.pckl')
a0 = si_bulk.cell[0, 0]
print('Lattice constant %.3f A\n' % a0)




Lattice constant 5.440 A



## Elastic constants


In [ ]:
from matscipy.elasticity import fit_elastic_constants
at = bulk("Si", cubic=True, a=a0)
at.calc = calc_asp_5000
_C, _C_err = fit_elastic_constants(at, symmetry='cubic', delta=1e-3)

C11, C12, C44 = np.array([_C[0, 0], _C[0, 1], _C[3, 3]]) / GPa
print('ACE_C11 =', C11, "GPa")
print('ACE_C12 =', C12, "GPa")
print('ACE_C44 =', C44, "GPa")

Fitting C_11
Strain array([-0.002, -0.001,  0.   ,  0.001,  0.002])
Stress array([-1.37125679, -1.21374749, -1.0566955 , -0.90009918, -0.74395661]) GPa
Cij (gradient) / GPa    :     156.82486618221415
Error in Cij / GPa      :     0.15561340651997607
Correlation coefficient :     0.9999985230885226
Setting C11 (1) to 0.978824 +/- 0.000971


Fitting C_21
Strain array([-0.002, -0.001,  0.   ,  0.001,  0.002])
Stress array([-1.17898989, -1.11744238, -1.0566955 , -0.99673201, -0.93753432]) GPa
Cij (gradient) / GPa    :     60.362151523740955
Error in Cij / GPa      :     0.2675519545274679
Correlation coefficient :     0.9999705314412973
Setting C21 (7) to 0.376751 +/- 0.001670


Fitting C_31
Strain array([-0.002, -0.001,  0.   ,  0.001,  0.002])
Stress array([-1.17898989, -1.11744238, -1.0566955 , -0.99673201, -0.93753432]) GPa
Cij (gradient) / GPa    :     60.36215152373957
Error in Cij / GPa      :     0.2675519545229254
Correlation coefficient :     0.9999705314412983
Updating C31 (7) 

In [ ]:

at.calc = calc_asp_5000
_C, _C_err = fit_elastic_constants(at, symmetry='cubic', delta=1e-3)

C11, C12, C44 = np.array([_C[0, 0], _C[0, 1], _C[3, 3]]) / GPa
print('ASP_C11 =', C11, "GPa")
print('ASP_C12 =', C12, "GPa")
print('ASP_C44 =', C44, "GPa")

Fitting C_11
Strain array([-0.002, -0.001,  0.   ,  0.001,  0.002])
Stress array([-1.37125679, -1.21374749, -1.0566955 , -0.90009918, -0.74395661]) GPa
Cij (gradient) / GPa    :     156.82486618221415
Error in Cij / GPa      :     0.15561340651997607
Correlation coefficient :     0.9999985230885226
Setting C11 (1) to 0.978824 +/- 0.000971


Fitting C_21
Strain array([-0.002, -0.001,  0.   ,  0.001,  0.002])
Stress array([-1.17898989, -1.11744238, -1.0566955 , -0.99673201, -0.93753432]) GPa
Cij (gradient) / GPa    :     60.362151523740955
Error in Cij / GPa      :     0.2675519545274679
Correlation coefficient :     0.9999705314412973
Setting C21 (7) to 0.376751 +/- 0.001670


Fitting C_31
Strain array([-0.002, -0.001,  0.   ,  0.001,  0.002])
Stress array([-1.17898989, -1.11744238, -1.0566955 , -0.99673201, -0.93753432]) GPa
Cij (gradient) / GPa    :     60.36215152373957
Error in Cij / GPa      :     0.2675519545229254
Correlation coefficient :     0.9999705314412983
Updating C31 (7) 

In [26]:

at = bulk("Si", cubic=True, a=a0)
at.calc = gap2b
_C, _C_err = fit_elastic_constants(at, symmetry='cubic', delta=1e-3)

C11, C12, C44 = np.array([_C[0, 0], _C[0, 1], _C[3, 3]]) / GPa
print('C11 =', C11, "GPa")
print('C12 =', C12, "GPa")
print('C44 =', C44, "GPa")

NameError: name 'gap2b' is not defined

## surface energy

In [ ]:
calc_asp_5000 = pyjulip.ACE1(f"asp5000.json")
surface_energy2  = find_surface_energy('Si', calc_asp_5000, a0=a0, surface='diamond110', fmax=1e-6)
print('Surface energy_2 = ', surface_energy2, '0.1*J/m^2')


PreconLBFGS:   0  15:04:22    -5221.661881       0.0000
PreconLBFGS:   0  15:04:22    -5214.257414       0.8009
PreconLBFGS:   1  15:04:23    -5215.043416       1.2438
PreconLBFGS:   2  15:04:23    -5215.550957       1.4308
PreconLBFGS:   3  15:04:23    -5215.742542       1.5070
PreconLBFGS:   4  15:04:23    -5215.983541       1.2537
PreconLBFGS:   5  15:04:23    -5216.094140       1.2337
PreconLBFGS:   6  15:04:23    -5216.316545       1.1508
PreconLBFGS:   7  15:04:23    -5216.503666       0.7583
PreconLBFGS:   8  15:04:23    -5216.600611       0.5364
PreconLBFGS:   9  15:04:23    -5216.668003       0.2803
PreconLBFGS:  10  15:04:23    -5216.688146       0.1841
PreconLBFGS:  11  15:04:23    -5216.700535       0.1255
PreconLBFGS:  12  15:04:23    -5216.707422       0.0864
PreconLBFGS:  13  15:04:23    -5216.713076       0.0706
PreconLBFGS:  14  15:04:23    -5216.715660       0.0505
PreconLBFGS:  15  15:04:23    -5216.716802       0.0513
PreconLBFGS:  16  15:04:23    -5216.718417      

In [ ]:
from ase.lattice.cubic import Diamond
from ase import Atom
import numpy as np

import ase.io, sys

import sys
import os


a0 = 5.44 # initial guess at lattice constant, cell will be relaxed below
tol = 3e-3 # maximum force following relaxtion [eV/A]
N = 3 # number of unit cells in each direction

bulk = Diamond(symbol='Si', latticeconstant=a0)

bulk.set_calculator(calc_asp_5000)

dyn = BFGS(bulk)
dyn.run(fmax=0.05)
bulk *= (N, N, N)
bulk_energy = bulk.get_potential_energy()
# print(bulk_energy)

def tetrahedral_interstitial_energy(bulk):
    Nat = bulk.get_number_of_atoms()
    int_struct = bulk.copy()
    int_struct.set_calculator(bulk.get_calculator())

    # add an atom to introduce an interstitial
    int_struct.append(Atom('Si', (0.001, 0.002, 5.44/2.0+0.003)))

    ase.io.write(sys.stdout, int_struct, format='extxyz')
    # relax atom positions, holding cell fixed

    int_ = BFGS(int_struct)
    int_.run(fmax=0.05)
    # int_struct = relax_atoms(int_struct, tol=tol, traj_file="model-"+"asp-test-interstitial-tetrahedral.opt.xyz")
    ase.io.write(sys.stdout, int_struct, format='extxyz')

    # compute formation energy as difference of bulk and int energies
    print('bulk cell energy', bulk_energy)
    print('interstitial cell energy', int_struct.get_potential_energy())
    e_form = int_struct.get_potential_energy() - bulk_energy*((Nat+1.0)/Nat)
    print('interstitial formation energy', e_form)
    return e_form

def dumbbell_interstitial_energy(bulk):
    Nat = bulk.get_number_of_atoms()
    int_struct = bulk.copy()
    int_struct.set_calculator(bulk.get_calculator())

    # add an atom to introduce an interstitial
    int_struct.append(Atom('Si', (-0.5, 0.5, 5.44/2.0+1.0)))
    p = int_struct.get_positions()
    p[149,0] -= 1.0
    p[149,1] += 1.0
    p[149,2] -= 0.5
    int_struct.set_positions(p)

    ase.io.write(sys.stdout, int_struct, format='extxyz')
    # relax atom positions, holding cell fixed
    int_ = BFGS(int_struct)
    int_.run(fmax=0.05)
    ase.io.write(sys.stdout, int_struct, format='extxyz')

    # compute formation energy as difference of bulk and int energies
    print('bulk cell energy', bulk_energy)
    print('interstitial cell energy', int_struct.get_potential_energy())
    e_form = int_struct.get_potential_energy() - bulk_energy*((Nat+1.0)/Nat)
    print('interstitial formation energy', e_form)
    return e_form


def hexagonal_interstitial_energy(bulk):
    Nat = bulk.get_number_of_atoms()
    int_struct = bulk.copy()
    int_struct.set_calculator(bulk.get_calculator())
    # add an atom to introduce an interstitial
    pos = int_struct.get_positions()
    int_pos = (pos[83,:] + pos[101,:] + pos[105,:] + pos[106,:] + pos[108,:] + pos[176,:])/6.0

    print("int_pos ", int_pos)
    int_struct.append(Atom('Si', int_pos))

    rot_mat = np.zeros( (3,3) )
    rot_mat[0,:] = (1.0/np.sqrt(2.0), -1.0/np.sqrt(2.0), 0.0)
    rot_mat[1,:] = (1.0/np.sqrt(6.0), 1.0/np.sqrt(6.0), -2.0/np.sqrt(6.0))
    rot_mat[2,:] = (1.0/np.sqrt(3.0), 1.0/np.sqrt(3.0), 1.0/np.sqrt(3.0))
    cell = np.dot(int_struct.get_cell(), rot_mat.T)
    int_struct.set_cell(cell, scale_atoms=True)

    int_struct.arrays['move_mask_3'] = np.ones( (len(int_struct), 3), dtype=int )
    int_struct.arrays['move_mask_3'][len(int_struct)-1,2] = 0

    ase.io.write(sys.stdout, int_struct, format='extxyz')
    # relax atom positions, holding cell fixed
    int_ = BFGS(int_struct)
    int_.run(fmax=0.05)
    ase.io.write(sys.stdout, int_struct, format='extxyz')

    # compute formation energy as difference of bulk and int energies
    print('bulk cell energy', bulk_energy)
    print('interstitial cell energy', int_struct.get_potential_energy())
    e_form = int_struct.get_potential_energy() - bulk_energy*((Nat+1.0)/Nat)
    print('interstitial formation energy', e_form)
    return e_form

properties = {'tetrahedral_interstitial_energy':
                tetrahedral_interstitial_energy(bulk),
              'hexagonal_interstitial_energy':
                hexagonal_interstitial_energy(bulk),
              'dumbbell_interstitial_energy':
                dumbbell_interstitial_energy(bulk)}


      Step     Time          Energy          fmax
BFGS:    0 22:15:06    -1305.418613        0.000000


/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:17: DeprecationWarning: Please use atoms.calc = calc
  bulk.set_calculator(calc_omp_300)
/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:26: FutureWarning: Please use len(self) or, if your atoms are distributed, self.get_global_number_of_atoms.
  Nat = bulk.get_number_of_atoms()
/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:28: DeprecationWarning: Please use atoms.calc
  int_struct.set_calculator(bulk.get_calculator())
/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:28: DeprecationWarning: Please use atoms.calc = calc
  int_struct.set_calculator(bulk.get_calculator())


217
Lattice="16.32 0.0 0.0 0.0 16.32 0.0 0.0 0.0 16.32" Properties=species:S:1:pos:R:3 energy=-35246.30254998397 free_energy=-35246.30254998397 pbc="T T T"
Si       0.00000000       0.00000000       0.00000000
Si       1.36000000       1.36000000       1.36000000
Si       2.72000000       2.72000000       0.00000000
Si       4.08000000       4.08000000       1.36000000
Si       2.72000000       0.00000000       2.72000000
Si       4.08000000       1.36000000       4.08000000
Si       0.00000000       2.72000000       2.72000000
Si       1.36000000       4.08000000       4.08000000
Si       0.00000000       0.00000000       5.44000000
Si       1.36000000       1.36000000       6.80000000
Si       2.72000000       2.72000000       5.44000000
Si       4.08000000       4.08000000       6.80000000
Si       2.72000000       0.00000000       8.16000000
Si       4.08000000       1.36000000       9.52000000
Si       0.00000000       2.72000000       8.16000000
Si       1.36000000       4.080000

/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:76: FutureWarning: Please use len(self) or, if your atoms are distributed, self.get_global_number_of_atoms.
  Nat = bulk.get_number_of_atoms()
/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:78: DeprecationWarning: Please use atoms.calc
  int_struct.set_calculator(bulk.get_calculator())
/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:78: DeprecationWarning: Please use atoms.calc = calc
  int_struct.set_calculator(bulk.get_calculator())


BFGS:    1 22:15:11   -35405.372472        1.132644
BFGS:    2 22:15:11   -35405.719881        0.382930
BFGS:    3 22:15:11   -35405.755831        0.299892
BFGS:    4 22:15:12   -35405.782721        0.189922
BFGS:    5 22:15:12   -35405.802211        0.166016
BFGS:    6 22:15:12   -35405.815040        0.087383
BFGS:    7 22:15:13   -35405.822021        0.095493
BFGS:    8 22:15:13   -35405.827794        0.097398
BFGS:    9 22:15:13   -35405.833967        0.098692
BFGS:   10 22:15:14   -35405.838464        0.087155
BFGS:   11 22:15:14   -35405.840735        0.079163
BFGS:   12 22:15:15   -35405.842192        0.062059
BFGS:   13 22:15:15   -35405.843620        0.051331
BFGS:   14 22:15:15   -35405.844992        0.048689
217
Lattice="11.539982668964456 6.662612100370246 9.422356393174693 -11.539982668964456 6.662612100370246 9.422356393174693 0.0 -13.325224200740491 9.422356393174693" Properties=species:S:1:pos:R:3:move_mask_3:I:3:forces:R:3 pbc="T T T"
Si      -0.00020551      -0.0001186

/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:49: FutureWarning: Please use len(self) or, if your atoms are distributed, self.get_global_number_of_atoms.
  Nat = bulk.get_number_of_atoms()
/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:51: DeprecationWarning: Please use atoms.calc
  int_struct.set_calculator(bulk.get_calculator())
/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/3740618316.py:51: DeprecationWarning: Please use atoms.calc = calc
  int_struct.set_calculator(bulk.get_calculator())


BFGS:    1 22:15:16   -35405.138395        1.997844
BFGS:    2 22:15:16   -35405.363159        0.973653
BFGS:    3 22:15:17   -35405.533010        0.710330
BFGS:    4 22:15:17   -35405.602119        0.717008
BFGS:    5 22:15:18   -35405.672024        0.745167
BFGS:    6 22:15:18   -35405.727298        0.582682
BFGS:    7 22:15:18   -35405.767109        0.317505
BFGS:    8 22:15:19   -35405.788011        0.290457
BFGS:    9 22:15:19   -35405.807265        0.290898
BFGS:   10 22:15:19   -35405.822356        0.230932
BFGS:   11 22:15:20   -35405.830433        0.189817
BFGS:   12 22:15:20   -35405.837123        0.184932
BFGS:   13 22:15:21   -35405.844532        0.247672
BFGS:   14 22:15:21   -35405.850336        0.183374
BFGS:   15 22:15:21   -35405.853891        0.112254
BFGS:   16 22:15:22   -35405.856353        0.121155
BFGS:   17 22:15:22   -35405.858566        0.111314
BFGS:   18 22:15:22   -35405.860117        0.076549
BFGS:   19 22:15:23   -35405.861047        0.047818
217
Lattice=

In [11]:
properties

{'tetrahedral_interstitial_energy': 3.7174793424492236,
 'hexagonal_interstitial_energy': 3.6348843255836982,
 'dumbbell_interstitial_energy': 3.6188298954148195}

In [ ]:
from ase.build import bulk
from ase.optimize import BFGS
from ase.optimize.precon import PreconLBFGS
from quippy.potential import Potential

si = bulk('Si', a=5.44, cubic=True)
sw_pot = calc_asp_5000
si.set_calculator(sw_pot)
e_bulk_per_atom = si.get_potential_energy()/len(si)

# call general purpose elastic constants calculator
#   using ASE Atoms and QUIP Potential
from matscipy.elasticity import fit_elastic_constants
Cij = fit_elastic_constants(si, optimizer=BFGS,
                            symmetry='cubic', logfile='-')
vac1 = si.copy()
vac1 *= (3, 3, 3)
half_cell = np.diag(vac1.cell)/2.
vac_atom = ((vac1.positions - half_cell)**2).sum(axis=1).argmin()
del vac1[vac_atom]

vac1.set_calculator(sw_pot)
vac1.rattle(0.01)
opt = PreconLBFGS(vac1) # big cell, use preconditioned minimiser
opt.run(fmax=1e-6)
e_vac = vac1.get_potential_energy() - e_bulk_per_atom*len(vac1)
print('SW vacancy formation energy', e_vac, 'eV')

      Step     Time          Energy          fmax
BFGS:    0 22:18:39    -1305.341029        0.216715
BFGS:    1 22:18:39    -1305.345381        0.134780
BFGS:    2 22:18:39    -1305.348129        0.000121
      Step     Time          Energy          fmax
BFGS:    0 22:18:39    -1305.391435        0.103538
BFGS:    1 22:18:39    -1305.392432        0.064931
BFGS:    2 22:18:39    -1305.393078        0.000014
      Step     Time          Energy          fmax
BFGS:    0 22:18:39    -1305.415076        0.000000
      Step     Time          Energy          fmax
BFGS:    0 22:18:39    -1305.412739        0.094371
BFGS:    1 22:18:39    -1305.413573        0.060314
BFGS:    2 22:18:39    -1305.414150        0.000013
      Step     Time          Energy          fmax
BFGS:    0 22:18:39    -1305.385235        0.180097


/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/562837793.py:8: DeprecationWarning: Please use atoms.calc = calc
  si.set_calculator(sw_pot)


BFGS:    1 22:18:40    -1305.388286        0.116317
BFGS:    2 22:18:40    -1305.390471        0.000123
Fitting C_11
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-4.21688374, -2.62045804, -1.04671294,  0.49631437,  2.00357037]) GPa
Cij (gradient) / GPa    :     155.57680622240602
Error in Cij / GPa      :     1.0229113894284063
Correlation coefficient :     0.9999351611440109
Setting C11 (1) to 0.971034 +/- 0.006385


Fitting C_21
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-2.40324999, -1.68699779, -1.04671294, -0.47362661,  0.04268665]) GPa
Cij (gradient) / GPa    :     61.05244473112136
Error in Cij / GPa      :     2.2817949533830526
Correlation coefficient :     0.9979113001058122
Setting C21 (7) to 0.381059 +/- 0.014242


Fitting C_31
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-2.40324999, -1.68699779, -1.04671294, -0.47362661,  0.04268665]) GPa
Cij (gradient) / GPa    :     61.052444731120055
Error in Cij / GPa      :  

/var/folders/s2/fw0z5_w155q9vszxj43yx2580000gn/T/ipykernel_21599/562837793.py:22: DeprecationWarning: Please use atoms.calc = calc
  vac1.set_calculator(sw_pot)


PreconLBFGS:   1  22:18:40   -35079.193832       0.3261
PreconLBFGS:   2  22:18:40   -35079.306393       0.2640
PreconLBFGS:   3  22:18:40   -35079.349060       0.2507
PreconLBFGS:   4  22:18:41   -35079.387372       0.2311
PreconLBFGS:   5  22:18:41   -35079.422520       0.2053
PreconLBFGS:   6  22:18:41   -35079.453717       0.1681
PreconLBFGS:   7  22:18:41   -35079.479718       0.1207
PreconLBFGS:   8  22:18:41   -35079.499253       0.0659
PreconLBFGS:   9  22:18:41   -35079.511564       0.0286
PreconLBFGS:  10  22:18:41   -35079.513679       0.0170
PreconLBFGS:  11  22:18:41   -35079.515578       0.0082
PreconLBFGS:  12  22:18:42   -35079.515732       0.0055
PreconLBFGS:  13  22:18:42   -35079.515810       0.0023
PreconLBFGS:  14  22:18:42   -35079.515825       0.0016
PreconLBFGS:  15  22:18:42   -35079.515836       0.0009
PreconLBFGS:  16  22:18:42   -35079.515839       0.0004
PreconLBFGS:  17  22:18:42   -35079.515840       0.0004
PreconLBFGS:  18  22:18:42   -35079.515840      

Passed direction which is not downhill. Aborting...: 1.258305e-05


PreconLBFGS:  37  22:18:44   -35079.516281       0.0157
PreconLBFGS:  38  22:18:44   -35079.516319       0.0155


/opt/anaconda3/envs/pycall_env/lib/python3.10/site-packages/ase/optimize/precon/lbfgs.py:340: UserWarning: Armijo linesearch failed, resetting Hessian and trying again
  warnings.warn(


PreconLBFGS:  39  22:18:45   -35079.516392       0.0150
PreconLBFGS:  40  22:18:45   -35079.516532       0.0141
PreconLBFGS:  41  22:18:45   -35079.516787       0.0121
PreconLBFGS:  42  22:18:45   -35079.517184       0.0082
PreconLBFGS:  43  22:18:45   -35079.517397       0.0078
PreconLBFGS:  44  22:18:45   -35079.517431       0.0082
PreconLBFGS:  45  22:18:45   -35079.517491       0.0087
PreconLBFGS:  46  22:18:45   -35079.517589       0.0092
PreconLBFGS:  47  22:18:45   -35079.517693       0.0094
PreconLBFGS:  48  22:18:46   -35079.517747       0.0095
PreconLBFGS:  49  22:18:46   -35079.517840       0.0105
PreconLBFGS:  50  22:18:46   -35079.518052       0.0119
PreconLBFGS:  51  22:18:46   -35079.518141       0.0106
PreconLBFGS:  52  22:18:46   -35079.518332       0.0072
PreconLBFGS:  53  22:18:46   -35079.518416       0.0054
PreconLBFGS:  54  22:18:46   -35079.518512       0.0042
PreconLBFGS:  55  22:18:46   -35079.518547       0.0041
PreconLBFGS:  56  22:18:46   -35079.518569      